In [50]:
%pip install tiktoken
%pip install openai
%pip install textract
%pip install numpy
%pip install pandas


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviou

# 라이브러리 임포트

In [2]:
import os
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract

# 논문 파일 리스트 구하기

In [3]:
import os

data_path = "data/paper"
pdf_files = [x for x in os.listdir(data_path)]

print(len(pdf_files))
print(pdf_files[:3])

20
['jkma-2023-66-1-50.pdf', 'jkma-2023-66-2-116.pdf', 'jkma-2023-66-3-191.pdf']


In [4]:
pdf_files = pdf_files[:2] 

# 파일을 조각(chunk)로 쪼개고 임베딩해 두기

In [5]:
import openai
openai.api_key = "sk-..."

In [48]:
import tiktoken

TEXT_EMBEDDING_CHUNK_SIZE = 2000
EMBEDDINGS_MODEL = "text-embedding-ada-002"
tokenizer = tiktoken.get_encoding("cl100k_base")

embedding_storage = []

# 긴 문자열을 최대 길이 chunk_size의 토큰들로 자르고, 토큰을 반환한다.
def chunks(text, chunk_size=TEXT_EMBEDDING_CHUNK_SIZE):
    """Yield successive chunk_size chunks from text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # 토큰 길이의 0.5 ~ 1.5 사이에서 문장의 끝을 찾는다.
        j = min(i + int(1.5 * chunk_size), len(tokens))
        while j > i + int(0.5 * chunk_size):
            # 디코딩 해서 마침표 혹은 줄 바꿈인지 확인한다.
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # 문장 끝을 못찾으면 chunk_size로 한다.
        if j == i + int(0.5 * chunk_size):
            j = min(i + chunk_size, len(tokens))
        yield tokens[i:j]
        i = j

def get_embeddings(text_array):
    embedding_objects = openai.Embedding.create(input=text_array, model=EMBEDDINGS_MODEL)["data"]
    return [embedding_object["embedding"] for embedding_object in embedding_objects]

    
def create_embeddings_for_text(text):
    # 최대길이 TEXT_EMBEDDING_CHUNK_SIZE의 토큰 리스트로 변환. 쪼개진 개별을 chunk라 한다.
    token_chunks = list(chunks(text))
    # 각 토큰 리스트를 text 리스트로 변환
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]

    # 전체 chunk들의 임베딩을 구하고
    embedding_chunks = get_embeddings(text_chunks)
    return text_chunks, embedding_chunks


def handle_file_string(filename, file_content_string):
    
    # 파일 내용에서 라인 변경, 더블 스페이스, 세미 콜론을 삭제
    clean_file_content_string = " ".join(file_content_string.replace("\n", "; ").replace(";", " ").split())
        
    try:
        text_chunks, embedding_chunks = create_embeddings_for_text(clean_file_content_string)
        print(f"[INFO] Embedded into {len(text_chunks)} chunks")
    except Exception as e:
        print("[handle_file_string] Error creating embedding: {}".format(e))

    for i, (text_chunk, embedding_chunk) in enumerate(zip(text_chunks, embedding_chunks)):
        embedding_storage.append({"embedding":embedding_chunk, "filename": filename, "chunk_index":i, "text":text_chunk })


In [49]:

# Process each PDF file and prepare for embedding
for i, pdf_file in enumerate(pdf_files):
    
    pdf_path = os.path.join(data_path, pdf_file)
    print(f"[INFO] {i}/{len(pdf_files)}. processing paper : {pdf_path}\n")
    
    text = textract.process(pdf_path, method='pdfminer')
    print(text.decode("utf-8")[0:200])
    print()

    handle_file_string(pdf_file, text.decode("utf-8"))
    print()
    print("============================================================")


[INFO] 0/2. processing paper : data/paper/jkma-2023-66-1-50.pdf

SPECIAL CONTRIBUTION 
J Korean Med Assoc 2023 January; 66(1):50-59

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.1.50

만성 코로나19 증후군 시대를 위한  
만성 코로나19 증후군 시대를 위한  
보건의료 대응 방안

[INFO] Embedded into 7 chunks

[INFO] 1/2. processing paper : data/paper/jkma-2023-66-2-116.pdf

FOCUSED ISSUE OF THIS MONTH 
J Korean Med Assoc 2023 February; 66(2):116-122

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.2.116

면역항암치료의 부작용 관리: 호흡계와 신경계를  
면역항암치료의 부작용 관리:

[INFO] Embedded into 5 chunks



In [41]:
from pprint import pprint

print(len(embedding_storage))
print(embedding_storage[0]["embedding"])
print(len(embedding_storage[0]["embedding"]))
print(embedding_storage[0]["filename"])
print(embedding_storage[0]["chunk_index"])
print(embedding_storage[0]["text"])

12
[-0.0028462682384997606, -0.011910849250853062, 0.03311040997505188, -0.02655843086540699, -0.013211808167397976, 0.01662261225283146, -0.012052404694259167, 0.0060936096124351025, -0.032813817262649536, 0.02955130860209465, -0.009699894115328789, 0.027245983481407166, -0.02282407321035862, 0.025318138301372528, -0.010737963952124119, -0.013872398063540459, 0.04036341980099678, -0.01666305586695671, 0.00781923346221447, -0.020208673551678658, -0.006767681334167719, -0.004465725738555193, -0.03222063556313515, 0.0005274612340144813, -0.0016733832890167832, 0.0263427272439003, 0.0007275762618519366, -0.018536975607275963, -0.0037141358479857445, -0.00012322876136749983, 0.012746698223054409, -0.01609683595597744, -0.012059145607054234, 0.012409662827849388, 0.011075000278651714, 0.01725623942911625, 0.018253864720463753, -0.01708098128437996, 0.017916830256581306, -0.017741570249199867, 0.014344248920679092, 0.010872778482735157, 0.004111838061362505, 0.0016540037468075752, -0.0048667

# 가장 가까운 조각(chunk) 구하기

In [52]:
from openai.embeddings_utils import distances_from_embeddings, indices_of_nearest_neighbors_from_distances

def find_closest_n_index(embedding_storage, embedded_query, n=3):
    embeddings = [entity["embedding"] for entity in embedding_storage]
    distances = distances_from_embeddings(embedded_query, embeddings, distance_metric="cosine")
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
    return indices_of_nearest_neighbors[:n]


def find_closest_n_chunk(embedding_storage, query, n=3):
    embedded_query = get_embeddings(query)[0]
    closest_indexes = find_closest_n_index(embedding_storage, embedded_query, n)
    return [embedding_storage[i] for i in closest_indexes]


In [53]:
question = "코로나19 증후군의 증상은?"
queryed = find_closest_n_chunk(embedding_storage, question, n=1)[0]

In [54]:
print(queryed["filename"])
print(queryed["chunk_index"])
print(queryed["text"])


jkma-2023-66-1-50.pdf
3
 또한 입원을 필요로 하는 중증 COVID-19 환자의 증후군과 관련한 이상신호를 실시간으로 모니터링하고, 위 경우 특히 급성기의 심장·폐 손상에 따른 심폐후유증, 심부 험 가능성이 있는 인자에 대해서는 보다 즉각적이고 면밀 정맥혈전증 및 폐색전증을 포함하는 혈전증, 뇌신경계 질환 한 분석을 통해 만성 코로나19 증후군에 대한 능동적인 관 등의 후유증과 연관성을 보이는 것으로 나타났다[35-37]. 찰 및 관리체계 구축을 도모할 수 있다. 이는 궁극적으로 이처럼 만성 코로나19 증후군 발생에 상대적으로 더 취약한 COVID-19로 인한 사회경제적 부담 완화에 기여한다. 또한 고위험군을 파악하여 환자·계층 맞춤형 만성 코로나19 증 COVID-19 회복 환자 등록 DB를 중앙암등록본부 DB, 통계 후군 예방 및 관리 방안 마련이 시급하다. 청 DB 등 타기관의 자료와 광범위하게 연계함으로써 그 활 54 대한의사협회지 Kim HJ · Song J · Park SM·Proactive strategies against long COVID 용성과 유용성을 더 확대할 수 있다[38]. 실제로 올해 6월부터 질병관리청과 국민건강보험공단의 주도 하에 COVID-19 빅데이터를 활용한 민·관 공동 방역 만성 코로나19 증후군 대응을 위한 정책적 방향 정책 과제가 진행되고 있다. 주요 과제들로서 만성 코로나19 국내에서는 일찍이 빅데이터의 중요성을 인지하여 다양 증후군 증상과 예후, 기저질환과의 관계, COVID-19가 취 한 데이터를 수집하고 정비함으로써 대규모의 DB를 구축 약계층에 미치는 영향 등에 대한 평가를 수행하고 있다. 이 한 바 있다. 다만, 이러한 방대한 DB는 서로 유기적으로 연 렇듯 여러 정부기관의 보건의료 빅데이터를 결합하여 개인 결되기 보다는 각각 독립적으로 운영되고 있는 것이 현실이 의 생활습관, 신체지표, 환경 등의 데이터를 연계하여 다양 다. 섬처럼 고립된 형태의 DB에 비하여 이들을 결합하여 유 한 개인 

# 구한 조각 내용으로 문의하기

In [45]:
prompt = f"""You will be provided with text delimited by triple quotes and question. answer the question based on the provided text

Text:
```{queryed["text"]}```

Question:
```{question}```
"""


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are helpful QnA engine."},
        {"role": "user", "content": prompt},
    ],
    temperature=0,
)

print(response['choices'][0]['message']['content'])

코로나19 증후군의 증상은 다양하며, 급성기의 심장·폐 손상에 따른 심폐후유증, 혈전증, 뇌신경계 질환 등과 관련이 있다고 나와있습니다.
